In [4]:
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import json
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)
gme = yf.Ticker("GME")
print(gme)
gme_info = gme.info
gme_share_price_data = gme.history(period="max")
gme_head_data = gme_share_price_data.head()
gme_share_price_data.reset_index(inplace=True)
print(gme_share_price_data.head())
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
data  = requests.get(url).text
soup = BeautifulSoup(data, 'html5lib')
table = soup.find_all("tbody")[1]
# Collect all rows in a list first
data = []

for row in table.find_all('tr'):
    col = row.find_all('td')
    if col:
        date = col[0].text.strip()
        revenue = col[1].text.strip()
        data.append({"Date": date, "Revenue": revenue})

# Convert the list of dictionaries to a DataFrame in one go
gme_revenue = pd.DataFrame(data)
# Remove commas and dollar signs from the 'Revenue' column
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"", regex=True)

gme_revenue.dropna(inplace=True)

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

# Print the updated DataFrame
print(gme_revenue.tail())

yfinance.Ticker object <GME>
                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620129  1.693350  1.603296  1.691667  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683250  11021600   
2 2002-02-15 00:00:00-05:00  1.683250  1.687458  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615920  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  
          Date Revenue
57  2006-01-31    1667
58  2005-10-31     534
59  2005-07-31     416
60  2005-04-30     475
61  2005-01-31     709


In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Define the plotting function
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

# Call the function to make the graph
make_graph(gme_share_price_data, gme_revenue, 'GME Stock Data and Revenue')

C:\Users\Jerry Hunter\AppData\Local\Temp\ipykernel_12468\3063753910.py:8: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\Jerry Hunter\AppData\Local\Temp\ipykernel_12468\3063753910.py:9: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

